In [ ]:
import os
current_path = os.getcwd()
print(current_path)
os.chdir("..")
current_path = os.getcwd()
print(current_path)

In [ ]:
# 第二个实验测试
# 我已经选择了合适的学习率, 打算跑1000次然后记录图片并且查看相应的降噪成果, 没啥东西就用这个跑
# 模型相关
import torch
from models.DIP_2D import DIP_2D
import pytorch_lightning as pl
import numpy as np

# 画图相关
import matplotlib.pyplot as plt

# 文件读写相关
import csv
from config.config import *
import pandas as pd
import csv

# 自定义函数
from utils.pre_utils import *

from functools import partial

import datetime

In [ ]:
path_input = "data/noisy_images/uniform_noise.npy"
path_ground_truth = "data/ground_truth/ground_truth.npy"
path_target = "data/corrupted_images/BSREM_it30.npy"

In [ ]:
image_net_input = np.load(path_input)

# image_net_input_scale,param1_scale_im_net,param2_scale_im_net = rescale_imag(image_net_input,"standardization") 
# image_net_input_torch = torch.Tensor(image_net_input_scale)

image_net_input_torch = torch.Tensor(image_net_input)
image_net_input_torch = image_net_input_torch.view(1,1,PETImage_shape[0],PETImage_shape[1],PETImage_shape[2])
image_net_input_torch = image_net_input_torch[:,:,:,:,0]

ground_truth = np.load(path_ground_truth)

image_corrupt = np.load(path_target)
image_corrupt_input_scaled,param1_scale_im_corrupt,param2_scale_im_corrupt = rescale_imag(image_corrupt,"standardization") # 标准化图片, 减去平均值，除以标准差，参数1是mean，参数2是std
image_corrupt_torch = torch.Tensor(image_corrupt_input_scaled)
image_corrupt_torch = image_corrupt_torch.view(1,1,PETImage_shape[0],PETImage_shape[1],PETImage_shape[2])
image_corrupt_torch = image_corrupt_torch[:,:,:,:,0]

plt.imshow(ground_truth, cmap='gray')
plt.show()  

In [ ]:

 # 加载数据
train_dataset = torch.utils.data.TensorDataset(image_net_input_torch,image_corrupt_torch)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1) 

# 加载模型
model = DIP_2D(param1_scale_im_corrupt, param2_scale_im_corrupt, config,'data/Algo/',
            "nested",all_images_DIP="False",global_it=-100, suffix='aaa',last_iter=-1,ground_truth=ground_truth,target=image_corrupt,initial_param='kaiming_norm')
# model.init_weights()
model_class = DIP_2D

#定义tensorboard
checkpoint_simple_path = os.getcwd() + '/test2_logs'
# experiment = 24
name=str(datetime.datetime.now())

logger = pl.loggers.TensorBoardLogger(save_dir=checkpoint_simple_path,name=name)#version=format(experiment), name=name)
trainer = pl.Trainer(max_epochs=config["sub_iter_DIP"],log_every_n_steps=1,logger=logger)#, callbacks=[checkpoint_callback, tuning_callback, early_stopping_callback], logger=logger,gpus=gpus, accelerator=accelerator, profiler="simple")

# 训练模型
trainer.fit(model, train_dataloader)
out = model(image_net_input_torch)

image_out = out.view(PETImage_shape[0],PETImage_shape[1],PETImage_shape[2]).detach().numpy()
image_concat = np.concatenate((image_corrupt, destand_numpy_imag(image_out,param1_scale_im_corrupt,param2_scale_im_corrupt)), axis=1)
image_reversed =np.max(image_concat)-image_concat

plt.imshow(image_reversed, cmap='gray')
plt.show()  

In [1]:
from test import *

psnr_list,mse_gt_list = main()

/home/xzhang/miniconda3/envs/dip/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/xzhang/miniconda3/envs/dip/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/xzhang/miniconda3/envs/dip/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1292: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(

   | Name       | Type       | Params
-------------------------------------------

/home/xzhang/Documents/我的模型/experiments
current working directory switched to /home/xzhang/Documents/我的模型
staaaaaaaaaaand
Number of params: 535251
Epoch 39:   0%|          | 0/1 [00:00<00:00, 980.66it/s, loss=0.166, v_num=0]  0
Succesfully save in: data/output_images/output_aaa_39.img
Epoch 79:   0%|          | 0/1 [00:00<00:00, 671.30it/s, loss=0.0997, v_num=0]  0
Succesfully save in: data/output_images/output_aaa_79.img
Epoch 119:   0%|          | 0/1 [00:00<00:00, 704.21it/s, loss=0.08, v_num=0]    0
Succesfully save in: data/output_images/output_aaa_119.img
Epoch 159:   0%|          | 0/1 [00:00<00:00, 693.85it/s, loss=0.0612, v_num=0]  0
Succesfully save in: data/output_images/output_aaa_159.img
Epoch 199:   0%|          | 0/1 [00:00<00:00, 909.63it/s, loss=0.0494, v_num=0]  0
Succesfully save in: data/output_images/output_aaa_199.img
Epoch 239:   0%|          | 0/1 [00:00<00:00, 924.06it/s, loss=0.0389, v_num=0]  0
Succesfully save in: data/output_images/output_aaa_239.img
Epoch 

In [2]:
filename = 'data/results/statistics/' + str(datetime.datetime.now()).split(' ')[0] +'.csv'
save_csv(filename,psnr_list,mse_gt_list)

In [3]:

tb = read_csv(filename)
# sns.relplot(x=)

   iters       psnr       mse_gt
0      0  15.545695  4826.700598
1      1  15.783134  4404.932145
2      2  18.142100  2438.853499
3      3  17.514765  2099.006888
4      4  15.849874  1903.451395
            iters        psnr       mse_gt
count  400.000000  400.000000   400.000000
mean   199.500000   26.144598   887.303635
std    115.614301    1.928070   316.167230
min      0.000000   15.545695   574.990220
25%     99.750000   26.260973   768.287932
50%    199.500000   26.699021   871.519780
75%    299.250000   27.027078   962.026121
max    399.000000   27.899275  4826.700598
